In [1]:
!pip install hyperopt

In [16]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1) 

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax'),
  ])

#train_and_predict( get_cnn_v5(input_shape, num_classes) )

In [8]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.7889 - accuracy: 0.2581 - val_loss: 1.1432 - val_accuracy: 0.6538
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.9652 - accuracy: 0.6927 - val_loss: 0.3917 - val_accuracy: 0.8750
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4781 - accuracy: 0.8504 - val_loss: 0.1147 - val_accuracy: 0.9650
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3128 - accuracy: 0.9045 - val_loss: 0.1002 - val_accuracy: 0.9698
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2234 - accuracy: 0.9315 - val_loss: 0.0428 - val_accuracy: 0.9882


0.9598639455782313

In [14]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1663 - accuracy: 0.9599


0.9598639607429504

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),
    
    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [27]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.3050508863225766, 'dropout_cnn_block_three': 0.3017293199424828, 'dropout_cnn_block_two': 0.4815538903683155, 'dropout_dense_block_one': 0.5007519532034042, 'dropout_dense_block_two': 0.3164067762054826}
accuracy=0.9625850319862366
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.39125206981284566, 'dropout_cnn_block_three': 0.3705548594844743, 'dropout_cnn_block_two': 0.4497353248700722, 'dropout_dense_block_one': 0.6462429437562552, 'dropout_dense_block_two': 0.6637800437699417}
accuracy=0.9247165322303772
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.40196038056482736, 'dropout_cnn_block_three': 0.30336767389346336, 'dropout_cnn_block_two': 0.3721429691026839, 'dropout_dense_block_one': 0.5929214863693029, 'dropout_dense_block_two': 0.37845846373174014}
accuracy=0.9585034251213074
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.4648307475297264, 'dropout_cnn_block_three': 0.35262960069545624, 'dropout_cnn_block_two': 0.41015279398